# 基于GLM的文本摘要

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
ds = Dataset.load_from_disk("../data/nlpcc_2017")
ds 

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [3]:
ds = ds.train_test_split(100, seed=42)
ds 

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 100
    })
})

In [4]:
ds["train"][0]

{'title': '组图:黑河边防军人零下30℃户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。',
 'content': '中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30℃严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30℃严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练'}

对于高版本的Transformers加载会报错，需要修改源码 \
文件地址 "你的放模型的地方"\THUDM\glm-large-chinese\tokenization_glm.py \
231行 super().__init__(**kwargs) 移动至 235行，放至self.sp_model.Load(vocab_file)的后面一行

In [7]:
tokenizer = AutoTokenizer.from_pretrained("../THUDM/glm-large-chinese", trust_remote_code=True)
tokenizer 

GLMChineseTokenizer(name_or_path='../THUDM/glm-large-chinese', vocab_size=50000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'eos_token': '<|endoftext|>', 'unk_token': '[UNK]', 'pad_token': '<|endoftext|>', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<|startofpiece|>', '<|endofpiece|>', '[gMASK]', '[sMASK]']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	50000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50001: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50002: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50003: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50004: AddedToken("[UNUSED1]", rstrip=False, lstrip=False, single_wor

In [9]:
def process_function(examples):
    contents = ["摘要生成：\n" + e + tokenizer.mask_token for e in examples["content"]]
    inputs = tokenizer(contents, max_length=384, truncation=True, padding="max_length", return_tensors="pt")
    inputs = tokenizer.build_inputs_for_generation(inputs, targets=examples["title"], max_gen_length=64, padding=True)
    return inputs

tokenized_ds = ds.map(process_function, batched=True, remove_columns=ds["train"].column_names)
print(tokenized_ds)

Map:   0%|          | 0/4900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [10]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'[CLS] 摘要生成: 中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30°C严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30°C严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练[MASK]<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endofte

In [13]:
print(tokenized_ds["train"][0]["position_ids"])

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained("../THUDM/glm-large-chinese", trust_remote_code=True)

In [18]:
train_args = Seq2SeqTrainingArguments(
    output_dir="result_glm",
    per_device_eval_batch_size=4,
    per_gpu_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    num_train_epochs=1
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer)
)

C:\Users\柒\AppData\Local\Temp\ipykernel_6564\3559992275.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
trainer.train()

  0%|          | 0/76 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
input_text = ds["test"][-1]["content"]
inputs = tokenizer("摘要生成：\n" + input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
inputs = inputs.to("cuda")
output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
tokenizer.decode(output[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'[CLS] 摘要生成: 中国证券网讯(记者严政)中国重工6月14日晚间公告称,公司日前接到控股股东中国船舶重工集团公司(简称“中船重工”)通知,中船重工拟对其自身相关业务进行调整,部分业务涉及到公司。其中,公司拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司。公司上述对外投资的方案不构成重大资产重组,也不涉及公司发行股份。中国重工表示,目前方案还需进一步论证,存在不确定性。为保证公平信息披露,维护投资者利益,避免造成公司股价异常波动,经公司申请,自6月12日下午开市起公司股票停牌。同时公司将与中船重工保持密切联系,尽快确认是否进行上述事项,并于股票停牌之日起5个工作日内(含停牌当日)公告事项进展情况。[MASK]<|endoftext|> <|startofpiece|> 公司停牌,已审议中船重工对其相关业务进行调整 <|endofpiece|>'

In [23]:
import torch

model = model.eval()


def predict_test():
    predict = []
    with torch.inference_mode():
        for d in ds["test"]:
            inputs = tokenizer("摘要生成：\n" + d["content"] + tokenizer.mask_token, return_tensors="pt")
            inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
            inputs = inputs.to("cuda")
            output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
            predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
            print("curID:", len(predict))
    return predict

In [24]:
result = predict_test()
result

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


curID: 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


curID: 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


curID: 98


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


curID: 99
curID: 100


['另 外媒:IS已公布1400人名单,大部分为美国和英国官员,其中包括外交人员和国防官员。',
 '安徽宿松县两名公职人员涉毒被通报:分别吸食由其提供毒品和行政拘留13天。',
 '黑龙江:每天平均有近千个“伪基站”用户来电投诉;黑龙江电信举报手机信号异常信号强度超标',
 '从明天起,北京市将实行实施新一轮机动车尾号轮换,为2和7,周一限行;周五限行,本周五、本周六北京、天津同步限行。',
 '哈尔滨:一男子与妻子有外遇,被母亲与妻子打死后撕扯其上衣,女子情绪失控,辱骂其父,辅警劝解后离场。',
 '苏州吴江初三中考泄题,考试前一天,考题已在网上疯传,称涉嫌泄题违纪被停职,被指泄题者曾出现在吴江市高中保送名单。',
 '承德通报承德广播电视台2名处级干部严重违纪被开除党籍、开除公职,其涉嫌犯罪问题移送司法机关依法处理。',
 '美男李佩斯开微博称“自己是中文官,手写汉字向网友问好',
 '“伊斯兰国”在叙利亚等部分国家攫取小麦和农产品,并控制部分小麦和农产品出口。',
 '国新办:渤海有西北风9级,阵风10-11级。',
 '中共中央政治局委员、上海市委书记韩正在沪会见缅甸全国民主联盟主席昂山素季并介绍上海发展情况。',
 '湖南三湘第一寿星:已活122岁,生前摇蒲扇扇凉,自言活到150岁',
 '杭州市大关街道通报:正科级干部包某某被举报涉嫌强奸他人,后与他人通奸。',
 '腰缠万贯”,',
 '武汉一男生摆摊“招聘”女友,现场有人围观拍照,称其为作秀',
 '江汉平原今晨出现霾,预计未来24小时内出现下列条件之一,并可能持续,',
 '台湾艺人朱孝天晒情侣拼图疑似公布恋情,称即将来临的冬不再令人感到“黑暗”。',
 '山东莱州:莱州市首光伏发电工程建成发电,每月能赚6000元',
 '陕西一8岁孙女失踪4天后被发现吊桥,因孩子调皮奶奶打了孩子一下。',
 '郑州5月商品房销售均价达10243元/平方米,涨幅29.04%。',
 '英特尔计划分四批发行70亿美元国债,期限最长的是规模为20亿美元的30年期债劵;',
 '合肥:合肥夫妻遭同事出轨,妻子谎称出轨被灌醉,丈夫愤怒提出离婚。',
 '江西瑞昌市政协党组成员、副主席徐新民涉嫌严重违纪,目前正接受组织调查。',
 '曹某涉嫌编造虚假恐怖信息被拘,要8万元给医院报爆炸地点,称其母亲生病要置其母身死地;后经研

In [25]:
from rouge_chinese import Rouge

rouge = Rouge()

docode_preds = [" ".join(p) for p in result]
decode_labels = [" ".join(l) for l in ds["test"]["title"]]
scores = rouge.get_scores(docode_preds, decode_labels, avg=True)
{
    "rouge-1": scores["rouge-1"]["f"],
    "rouge-2": scores["rouge-2"]["f"],
    "rouge-l": scores["rouge-l"]["f"],
}

{'rouge-1': 0.42880108425586594,
 'rouge-2': 0.23254478893676173,
 'rouge-l': 0.3342818459424747}